# NER модель 
# для удаления названий компаний из текстов

Названия компаний, в основном, это идентификаторы на которых переобучается модель. Разработаем автоматический фильтр названий компаний для предобработки набора данных.

Сделаем объединение результатов с двух моделей: эвристической и BERT.

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import warnings
from collections import namedtuple

import torch

warnings.filterwarnings("ignore")


NerItem = namedtuple("NerItem", ["word", "index_start", "index_end"])

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [3]:
TXT = "IVAT it  IVA Technologies - один из крупных игроков российского \
IT-рынка разработчик экосистемы корпоративных коммуникаций IVA.  \
В экосистему IVA входят: корпоративный мессенджер IVA Connect \
платформа видеоконференцсвязи IVA MCU облачная платформа видеоконференцсвязи \
«ВКурсе» помощник основанный на технологии искусственного интеллекта IVA GPT \
система управления и мониторинга IVA MS сервер корпоративной телефонии \
IVA CS серия IP-телефонов видеотерминалы IVA Room и IVA Largo пограничный \
контроллер сессий IVA SBC и другие продукты."

## Модель1 Эвристические правила

Простая модель основана на эвристрических правилах обработки текстовых строк.

In [4]:
import re


class NERFilter:
    """Heuristics NER Model"""

    def __init__(self):
        self._low_freq_words = []

    # Удаляем подстроки с большой буквы внутри кавычек «»
    def _rule_double_angle_quotation(self, s, do_filter=True):
        result = s
        pattern = '[ (]«[А-ЯЁA-Z][^«»]*»'
        matches = re.findall(pattern, s)

        if do_filter:
            for m in matches:
                result = result.replace(m, '')

            return result

        return matches

    # Удаляем низкочастотные подстроки с большой буквы
    def _rule_low_freqs_with_caps(self, s):
        result = []
        for w in s.split(' '):
            if w not in self._low_freq_words:
                result.append(w)

        return ' '.join(result)

    def fit(self, corpus, important_words, level=10):
        """
        Args:
            corpus — список строк;
            importantWords — список слов для сохранения
                    (такие слова не будут удаляться);
            level — уровень кол-ва появления слова в текстах,
                    чтобы его отфильтровать
                    (слова с большой буквы с частотностью ниже level
                    будут удалены при вызове метода filter).
        """
        freqs = {}

        # Get frequencies of words:
        for txt in corpus:
            for w in txt.split(' '):
                # print('"'+w+'"')
                if len(w) > 1 \
                        and w[0].upper() == w[0] \
                        and w.lower() not in important_words \
                        and '-' not in w:
                    if w in freqs:
                        freqs[w] = freqs[w] + 1
                    else:
                        freqs[w] = 1

        # Write results:

        for k, v in freqs.items():
            if v < level:
                self._low_freq_words.append(k)

        return self._low_freq_words

    def filter(self, s):
        """
        Метод для фильтрации названий из текста.
        """
        result = self._rule_double_angle_quotation(s)
        result = self._rule_low_freqs_with_caps(result)
        return result.replace('  ', ' ')

    def get_ner_results(self, s):
        """
        Метод для получения результатов анализа NER.
        """
        results = {}

        for m in self._rule_double_angle_quotation(s, False):
            results[m] = True

        for w in self._low_freq_words:
            results[w] = True

        return list(map(lambda it: NerItem(it, -1, -1), results))

In [5]:
corpus_example = [TXT,]
important_words_example = ['gpt',]

model_ner = NERFilter()
model_ner.fit(corpus_example, important_words_example, level=15)
heuristic_ner_results = model_ner.get_ner_results(TXT)
print('Entities:\n', heuristic_ner_results)
print('Filtered text:\n', model_ner.filter(TXT))

Entities:
 [NerItem(word=' «ВКурсе»', index_start=-1, index_end=-1), NerItem(word='IVAT', index_start=-1, index_end=-1), NerItem(word='IVA', index_start=-1, index_end=-1), NerItem(word='Technologies', index_start=-1, index_end=-1), NerItem(word='IVA.', index_start=-1, index_end=-1), NerItem(word='Connect', index_start=-1, index_end=-1), NerItem(word='MCU', index_start=-1, index_end=-1), NerItem(word='«ВКурсе»', index_start=-1, index_end=-1), NerItem(word='MS', index_start=-1, index_end=-1), NerItem(word='CS', index_start=-1, index_end=-1), NerItem(word='Room', index_start=-1, index_end=-1), NerItem(word='Largo', index_start=-1, index_end=-1), NerItem(word='SBC', index_start=-1, index_end=-1)]
Filtered text:
 it - один из крупных игроков российского IT-рынка разработчик экосистемы корпоративных коммуникаций В экосистему входят: корпоративный мессенджер платформа видеоконференцсвязи облачная платформа видеоконференцсвязи помощник основанный на технологии искусственного интеллекта GPT сис

Мы получили NER модель, которую можно использовать для предобработки текстов описаний компаний в развитом ПО, которое будет использовать ML модель.

## Модель2 NER BERT

Используем предобученную модель BERT based: https://huggingface.co/webmichaelnosenko/rust-bert-base-NER

In [6]:
from transformers import pipeline

In [7]:
def join_results(bert_results, txt):
    """
    Функция для объединения результатов infer от модели BERT 
    в законченные слова.
    """
    results = []

    if len(bert_results) > 0:
        start = bert_results[0]['start']
        end = bert_results[0]['end']
        for result in bert_results:
            if result['start'] - end > 1:
                results.append(NerItem(txt[start:end + 1], start, end))
                start = result['start']

            if txt[result['end']].isalpha():
                end = result['end']
            else:
                end = result['end'] - 1

        results.append(NerItem(txt[start:end + 1], start, end))

    return results


def get_bert_results(txt, level_score, path_bert, verbose=True):
    """
    Функция для получения результатов infer от модели BERT
    """
    ner_pipe = pipeline("ner", model=path_bert, device=DEVICE)

    results = ner_pipe(txt)

    results = filter(
        lambda it: it['score'] > level_score and 'ORG' in it['entity'],
        results
    )

    results = list(results)

    if verbose:
        for result in results:
            print(result)

    processed_results = join_results(results, txt)

    return processed_results


def filter_by_bert_ner_results(ner_results, txt):
    """
    Функция для фильтрации текста с помощью результатов от BERT NER.
    """
    result = list(txt)
    null_character = chr(58368)  # null character

    for ner_item in ner_results:
        for i in range(ner_item.index_start, ner_item.index_end + 1):
            result[i] = null_character

    result = "".join(result).replace(null_character, '')
    return result


PATH_BERT = (
    r'H:\Инструменты\Windows\AI DataAnalysis ML NER etc\NER'
    r'\2024 предобученный BERT NER - webmichaelnosenko - rust-bert-base-NER'
)
LEVEL_SCORE = 0.5  # Используем результаты со score больше заданного уровня.
bert_ner_results = get_bert_results(TXT, LEVEL_SCORE, PATH_BERT)
print('Entities:\n', bert_ner_results)

print('Filtered text:\n', filter_by_bert_ner_results(bert_ner_results, TXT))

Some weights of the model checkpoint at H:\Инструменты\Windows\AI DataAnalysis ML NER etc\NER\2024 предобученный BERT NER - webmichaelnosenko - rust-bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda
3:61: W291 trailing whitespace


{'entity': 'B-ORG', 'score': np.float32(0.9943851), 'index': 4, 'word': 'IV', 'start': 9, 'end': 11}
{'entity': 'I-ORG', 'score': np.float32(0.99040914), 'index': 5, 'word': '##A', 'start': 11, 'end': 12}
{'entity': 'I-ORG', 'score': np.float32(0.9776227), 'index': 6, 'word': 'Technologies', 'start': 13, 'end': 25}
{'entity': 'I-ORG', 'score': np.float32(0.9517372), 'index': 89, 'word': 'IV', 'start': 123, 'end': 125}
{'entity': 'I-ORG', 'score': np.float32(0.8818024), 'index': 90, 'word': '##A', 'start': 125, 'end': 126}
{'entity': 'B-ORG', 'score': np.float32(0.679242), 'index': 103, 'word': 'IV', 'start': 142, 'end': 144}
{'entity': 'I-ORG', 'score': np.float32(0.6667436), 'index': 104, 'word': '##A', 'start': 144, 'end': 145}
{'entity': 'B-ORG', 'score': np.float32(0.63552064), 'index': 339, 'word': 'IP', 'start': 429, 'end': 431}
{'entity': 'I-ORG', 'score': np.float32(0.6018182), 'index': 363, 'word': '##A', 'start': 459, 'end': 460}
{'entity': 'B-ORG', 'score': np.float32(0.8383

## Модель3 NER Natasha

In [8]:
from natasha import (
    Segmenter,

    NewsEmbedding,
    NewsNERTagger,

    Doc
)

In [9]:
def split_doc_span_to_ner_items(doc_span):
    """The function for split a DocSpan to a list of NerItem."""
    results = [NerItem(word, -1, -1) for word in doc_span.text.split(" ")]
    return results


def get_natasha_results(txt, verbose=True):
    """The function for infer and get results."""
    results = []

    emb = NewsEmbedding()
    ner_tagger = NewsNERTagger(emb)
    segmenter = Segmenter()

    doc = Doc(txt)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)

    if verbose:
        print(doc.spans[:5])
        doc.ner.print()

    for doc_span in doc.spans:
        results += split_doc_span_to_ner_items(doc_span)

    return results


def filter_by_natasha_results(ner_results, txt):
    """
    Функция для фильтрации текста с использованием результатов от NER Natasha.
    """

    result = txt
    for ner_item in ner_results:
        result = result.replace(ner_item.word, "")

    return result

In [10]:
natasha_ner_results = get_natasha_results(TXT)
print('Entities:\n', natasha_ner_results)
print('Filtered text:\n', filter_by_natasha_results(natasha_ner_results, TXT))

[DocSpan(start=9, stop=25, type='ORG', text='IVA Technologies', tokens=[...]), DocSpan(start=64, stop=72, type='ORG', text='IT-рынка', tokens=[...]), DocSpan(start=123, stop=126, type='ORG', text='IVA', tokens=[...]), DocSpan(start=269, stop=275, type='ORG', text='ВКурсе', tokens=[...])]
IVAT it  IVA Technologies - один из крупных игроков российского IT-
         ORG─────────────                                       ORG
рынка разработчик экосистемы корпоративных коммуникаций IVA.  В 
─────                                                   ORG     
экосистему IVA входят: корпоративный мессенджер IVA Connect платформа 
видеоконференцсвязи IVA MCU облачная платформа видеоконференцсвязи 
«ВКурсе» помощник основанный на технологии искусственного интеллекта 
 ORG───                                                              
IVA GPT система управления и мониторинга IVA MS сервер корпоративной 
телефонии IVA CS серия IP-телефонов видеотерминалы IVA Room и IVA 
Largo пограничный контроллер 

## Объединим модели
Применим принцип голосования.

In [11]:
def combine_results_2(model1_results, model2_results):
    """
    Функция объединяет результаты двух моделей.
    """

    combined_results = []
    res1 = list(map(lambda it: it.word, model1_results))
    res2 = list(map(lambda it: it.word, model2_results))

    for it in res1:
        if it in res2:
            combined_results.append(it)

    return combined_results


def combine_results_3(model1_results, model2_results, model3_results):
    """
    Функция объединяет результаты трёх моделей.
    """

    combined_results_from_2_models = combine_results_2(model1_results,
                                                       model2_results)

    results_from_2_models = list(map(lambda it: NerItem(it, -1, -1),
                                     combined_results_from_2_models))

    combined_results = combine_results_2(results_from_2_models, model3_results)
    return combined_results


def filter_by_combined_results(ner_results_as_list_of_words, txt):
    """
    Функция для фильтрации текста с использованием результатов от NER моделей.
    """

    result = txt
    for word in ner_results_as_list_of_words:
        result = result.replace(word, '')

    return result

### Эвристическая модель + BERT NER

In [12]:
combined_ner_results = combine_results_2(heuristic_ner_results,
                                         bert_ner_results)

print('Entities:\n', combined_ner_results)
print('Filtered text:\n', filter_by_combined_results(combined_ner_results,
                                                     TXT))

Entities:
 ['IVA', 'Technologies']
Filtered text:
 T it    - один из крупных игроков российского IT-рынка разработчик экосистемы корпоративных коммуникаций .  В экосистему  входят: корпоративный мессенджер  Connect платформа видеоконференцсвязи  MCU облачная платформа видеоконференцсвязи «ВКурсе» помощник основанный на технологии искусственного интеллекта  GPT система управления и мониторинга  MS сервер корпоративной телефонии  CS серия IP-телефонов видеотерминалы  Room и  Largo пограничный контроллер сессий  SBC и другие продукты.


### Эвристическая модель + Natasha NER

In [13]:
combined_ner_results_variant2 = combine_results_2(heuristic_ner_results,
                                                  natasha_ner_results)

print('Entities:\n', combined_ner_results_variant2)
print('Filtered text:\n',
      filter_by_combined_results(combined_ner_results_variant2, TXT)
      )

Entities:
 ['IVA', 'Technologies']
Filtered text:
 T it    - один из крупных игроков российского IT-рынка разработчик экосистемы корпоративных коммуникаций .  В экосистему  входят: корпоративный мессенджер  Connect платформа видеоконференцсвязи  MCU облачная платформа видеоконференцсвязи «ВКурсе» помощник основанный на технологии искусственного интеллекта  GPT система управления и мониторинга  MS сервер корпоративной телефонии  CS серия IP-телефонов видеотерминалы  Room и  Largo пограничный контроллер сессий  SBC и другие продукты.


### Эвристическая модель + BERT NER + Natasha NER

In [14]:
combined_ner_results_variant3 = combine_results_3(heuristic_ner_results,
                                                  natasha_ner_results,
                                                  bert_ner_results)

print('Entities:\n', combined_ner_results_variant3)
print('Filtered text:\n',
      filter_by_combined_results(combined_ner_results_variant3, TXT)
      )

Entities:
 ['IVA', 'Technologies']
Filtered text:
 T it    - один из крупных игроков российского IT-рынка разработчик экосистемы корпоративных коммуникаций .  В экосистему  входят: корпоративный мессенджер  Connect платформа видеоконференцсвязи  MCU облачная платформа видеоконференцсвязи «ВКурсе» помощник основанный на технологии искусственного интеллекта  GPT система управления и мониторинга  MS сервер корпоративной телефонии  CS серия IP-телефонов видеотерминалы  Room и  Largo пограничный контроллер сессий  SBC и другие продукты.


## Выводы

На прилагающемся скриншоте <a href="NERResults.png">NERResults.png</a> мы видим результаты фильтрации текста из примера:
- Слева Модель1 (работающая по эвристическим правилам). Модель агрессивно вычистила из текста не только название организации, но и все названия продуктов, но не тронула термины, например: «IP-телефонов», «IT-рынка».
- Справа Модель2 (BERT based). Модель удалила отдельные названия компании, но оставила артефакты: в названиях продуктов содержится и название компании, например: «IVA Connect». Также, модель удалила термин "IP-".
- В середине комбинированные результаты. Результаты сбалансированные: удалены все упоминания компании, но остались термины и названия продуктов.
- С помощью результатов от Модель3 (Natasha NER) полностью удалены упоминания названия компании в тексте, но удален также термин «IT-рынка» и название продукта «ВКурсе».
- Комбинации моделей дали схожие результаты.

Пользователь может выбрать нужный вариант NER модели или комбинацию в зависимости от требований к производительности решения.